In [ ]:
#!sudo apt-get install libpq-dev -y
!pip install psycopg2

# General psql queries

In [1]:
import psycopg2

In [3]:
#открываем соединение
conn_string="dbname='{}' user='{}' host='{}' password='{}'".format("titanik_db3", "student3",
                                                                  "ec2-18-202-17-151.eu-west-1.compute.amazonaws.com",
                                                                  "student3")
conn = psycopg2.connect(conn_string)
cursor = conn.cursor()

# Создаем таблицы в базе

In [6]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

data = pd.read_csv("../lecture_05_ml/notebooks/titanik.tsv", sep='|', index_col=0)
data['prename'] = data['name'].apply(lambda x: x.split()[1].strip("."))
data['pclass'] = data['pclass'].apply(str)
data['age'].fillna(0, inplace=True)
data['age'] = data['age'].apply(float)
data['fare'].fillna(0, inplace=True)
data['fare'] = data['fare'].apply(float)
data['embarked'].fillna("None", inplace=True)

lbl_embarked = LabelEncoder().fit(data['embarked'])
lbl_pclass = LabelEncoder().fit(data['pclass'])
lbl_prename = LabelEncoder().fit(data['prename'])
lbl_sex = LabelEncoder().fit(data['sex'])

data['embarked'] = lbl_embarked.transform(data['embarked'])
data['pclass'] = lbl_pclass.transform(data['pclass'])
data['prename'] = lbl_prename.transform(data['prename'])
data['sex'] = lbl_sex.transform(data['sex'])

data.head()

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest,prename
0,0,1,"Allen, Miss. Elisabeth Walton",0,29.0000,0,0,24160,211.3375,B5,3,2,NaN,"St Louis, MO",16
1,0,1,"Allison, Master. Hudson Trevor",1,0.9167,1,2,113781,151.5500,C22 C26,3,11,NaN,"Montreal, PQ / Chesterville, ON",13
2,0,0,"Allison, Miss. Helen Loraine",0,2.0000,1,2,113781,151.5500,C22 C26,3,NaN,NaN,"Montreal, PQ / Chesterville, ON",16
3,0,0,"Allison, Mr. Hudson Joshua Creighton",1,30.0000,1,2,113781,151.5500,C22 C26,3,NaN,135.0,"Montreal, PQ / Chesterville, ON",19
4,0,0,"Allison, Mrs. Hudson J C (Bessie Waldo Daniels)",0,25.0000,1,2,113781,151.5500,C22 C26,3,NaN,NaN,"Montreal, PQ / Chesterville, ON",20


In [9]:
# Создадим впомогательные таблицы и заполним их
# create_additional = '''
#         CREATE TABLE embarked (
#         embarked_id serial PRIMARY KEY,
#         embarked_type VARCHAR(10) NOT NULL UNIQUE
#         );
#         CREATE TABLE pclass (
#         pclass_id serial PRIMARY KEY,
#         pclass_type VARCHAR(10) NOT NULL UNIQUE
#         );
#         '''
# cursor.execute(create_additional)
conn.commit()

In [10]:
insert_query = '''INSERT INTO embarked (embarked_id, embarked_type) VALUES (%s, %s)''';
cursor.executemany(insert_query, list(zip(range(len(lbl_embarked.classes_)), lbl_embarked.classes_)))
conn.commit()
insert_query = '''INSERT INTO pclass (pclass_id, pclass_type) VALUES (%s, %s)''';
cursor.executemany(insert_query, list(zip(range(len(lbl_pclass.classes_)), lbl_pclass.classes_)))
conn.commit()

In [23]:
# #TODO Создайте таблицу для prename (и заполните её)
create_additional = '''
        CREATE TABLE prename (
        prename_id serial PRIMARY KEY,
        prename_name varchar(15) NOT NULL
        );
        '''
cursor.execute(create_additional)
conn.commit()

In [27]:
#TODO Создайте таблицу для пола
create_additional = '''
        CREATE TABLE sex (
        sex_id serial PRIMARY KEY,
        sex varchar(7) NOT NULL UNIQUE
        );
        '''
cursor.execute(create_additional)
conn.commit()


In [24]:
insert_query = '''INSERT INTO prename (prename_id, prename_name) VALUES (%s, %s)''';
cursor.executemany(insert_query, list(zip(range(len(lbl_prename.classes_)), lbl_prename.classes_)))
conn.commit()

In [28]:
insert_query = '''INSERT INTO sex (sex_id, sex) VALUES (%s, %s)''';
cursor.executemany(insert_query, list(zip(range(len(lbl_sex.classes_)), lbl_sex.classes_)))
conn.commit()

In [29]:
lbl_prename.classes_

array(['Billiard,', 'Brito,', 'Capt', 'Carlo,', 'Col', 'Cruyssen,', 'Don',
       'Dr', 'Gordon,', 'Impe,', 'Jonkheer', 'Khalil,', 'Major', 'Master',
       'Melkebeke,', 'Messemaeker,', 'Miss', 'Mlle', 'Mme', 'Mr', 'Mrs',
       'Ms', 'Mulder,', 'Palmquist,', 'Pelsmaeker,', 'Planke,', 'Rev',
       'Shawah,', 'Steen,', 'Velde,', 'Walle,', 'der', 'the', 'y'],
      dtype=object)

In [30]:
data.columns

Index(['pclass', 'survived', 'name', 'sex', 'age', 'sibsp', 'parch', 'ticket',
       'fare', 'cabin', 'embarked', 'boat', 'body', 'home.dest', 'prename'],
      dtype='object')

In [31]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1309 entries, 0 to 1308
Data columns (total 15 columns):
pclass       1309 non-null int64
survived     1309 non-null int64
name         1309 non-null object
sex          1309 non-null int64
age          1309 non-null float64
sibsp        1309 non-null int64
parch        1309 non-null int64
ticket       1309 non-null object
fare         1309 non-null float64
cabin        295 non-null object
embarked     1309 non-null int64
boat         486 non-null object
body         121 non-null float64
home.dest    745 non-null object
prename      1309 non-null int64
dtypes: float64(3), int64(7), object(5)
memory usage: 163.6+ KB


In [37]:
#TODO создать основную таблицу
create_additional = '''
        CREATE TABLE titanik (
        id serial PRIMARY KEY,
        pclass_id       INT references  pclass(pclass_id) ,
        survived     INT ,
        name         varchar(100),
        sex_id       INT references sex(sex_id),
        age          INT ,
        sibsp        INT ,
        parch        INT ,
        ticket       INT ,
        fare         INT ,
        cabin        varchar(10) ,
        boat         INT ,
        body         FLOAT ,
        embarked_id INT references  embarked(embarked_id) ,
        home     varchar(100) ,
        prename_id INT references prename(prename_id) 
        );
        '''
cursor.execute(create_additional)
conn.commit()

In [35]:
conn.commit()

In [ ]:
insert_query = '''INSERT INTO titanik  VALUES (%s, %s,%s, %s,%s, %s,%s, %s,%s, %s,%s)''';
cursor.executemany(insert_query, list(zip(range(len(lbl_prename.classes_)), lbl_prename.classes_)))
conn.commit()

In [ ]:
#TODO вставьте данные из таблицы

In [ ]:
#TODO Вопрос Кого больше мужчин или женщин?

In [ ]:
#TODO Вопрос Какой процент мужчин выжил и какой процент женщин?

In [ ]:
#TODO Что вы можете сказать выживаемости пассижиров разного класса и пола?